<a href="https://colab.research.google.com/github/xcsengody/DP/blob/master/xcsengody_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
root='/content/drive/My Drive/DP/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import pylab as pl
from pylab import rcParams
from collections import Counter
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import (SMOTE,SVMSMOTE,ADASYN)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from sklearn import metrics
from pprint import pprint
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve 
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
def plot_roc_curve(fpr, tpr):  
  plt.plot(fpr, tpr, color='orange', label='ROC')
  plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('Receiver Operating Characteristic (ROC) Curve')
  plt.legend()
  plt.show()

def predict(clf, X, y):
  prediction = clf.predict(X)

  accuracy = accuracy_score(y, prediction)
  print("Accuracy: %.2f%%" % (accuracy * 100.0))

  score = metrics.accuracy_score(y, prediction, normalized=False)
  print(score)

  print("\nClassification Report\n {}\n".format(classification_report(y, prediction)))

  print("\nConfusion-matrix\n {}\n".format(pd.crosstab(y, prediction, rownames=['Actual Species'], colnames=['Predicted Species'])))

  # ROC-AUC for binary class model performance evaluation
  proba = clf.predict_proba(X)
  proba = [p[1] for p in proba]
  print("\nROC-AUC: {}\n".format(roc_auc_score(y, proba)))
  
  fpr, tpr, thresholds = roc_curve(y.values, proba)
  plot_roc_curve(fpr, tpr)
  

  #score = metrics.f1_score(y_test, prediction, average=None)
  #print("F1 score: {}".format(score))

  #cv = cross_val_score(clf, X_train, y_train, cv=10, scoring='roc_auc')
  #print("Standard Cross-validation accuracy: %f (+/- %f)" % (cv.mean(), (cv.std()*2)))

  #skfold = StratifiedKFold(n_splits=10)

  #skfold_cv = cross_val_score(clf, X_train, y_train, cv=skfold, scoring='roc_auc')
  #print("Stratified K-fold Cross-validation accuracy of train set: %f (+/- %f)\n" % (skfold_cv.mean(), (skfold_cv.std()*2)))

  #skfold_cv = cross_val_score(clf, X_valid, y_valid, cv=skfold, scoring='roc_auc')
  #print("Stratified K-fold Cross-validation accuracy of validation set: %f (+/- %f)\n" % (skfold_cv.mean(), (skfold_cv.std()*2)))

  # Confusion-matrix usually used to evaluate the performance of a multiclass model.
  #conf_mat = confusion_matrix(y_test, prediction)
  #sns.heatmap(conf_mat,annot=True)
  #plt.title("Confusion-matrix")
  #plt.figure(figsize=(20,20))
  #plt.show()

In [0]:
pd.set_option('display.max_rows', 500);
pd.set_option('display.max_columns', None);
rcParams['figure.figsize'] = 15, 8;
sns.set(style="whitegrid");

In [0]:
#dataset=dataset.reset_index();
#dataset=dataset.set_index('index');

#joblib.dump(clf, 'randomforestmodel.sav')
#clf = joblib.load(root+'/Model/randomforestmodel_bin.sav')

In [0]:
train_dataset_path= root+"Dataset/dataset_4.csv";
valid_dataset_path= root+"Dataset/dataset_1.csv";

In [8]:
dataset_valid = pd.read_csv(valid_dataset_path, delimiter=',', encoding='utf-8', low_memory=False, skipinitialspace=True, skip_blank_lines=True, verbose=True);
del dataset_valid["Unnamed: 0"]

Tokenization took: 10522.79 ms
Type conversion took: 13726.11 ms
Parser memory cleanup took: 50.14 ms


In [9]:
print("Number of rows: {}\nNumber of features: {}".format(dataset_valid.shape[0],dataset_valid.shape[1]))

Number of rows: 699984
Number of features: 58


In [10]:
dataset_train = pd.read_csv(train_dataset_path, delimiter=',', encoding='utf-8', low_memory=False, skipinitialspace=True, skip_blank_lines=True, verbose=True);
del dataset_train["Unnamed: 0"]

Tokenization took: 5367.27 ms
Type conversion took: 7500.81 ms
Parser memory cleanup took: 19.70 ms


In [11]:
print("Number of rows: {}\nNumber of features: {}".format(dataset_train.shape[0],dataset_train.shape[1]))

Number of rows: 440042
Number of features: 58


In [12]:
y = dataset_train.attack_cat.apply(lambda x: 1 if x>0 else x)
X = dataset_train.drop('attack_cat', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

y_valid = dataset_valid.attack_cat.apply(lambda x: 1 if x>0 else x)
X_valid = dataset_valid.drop('attack_cat', axis=1)

print("Train set size: {}\nTest set size: {}\nValidation set size: {}".format(X_train.shape[0],X_test.shape[0],X_valid.shape[0]))

class_weights_arr = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights = {};

i=0
for cw in class_weights_arr:
  class_weights[i] = cw;
  i+=1

Train set size: 308029
Test set size: 132013
Validation set size: 699984


In [13]:
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 100, num = 10)]
max_features = ['auto', 'sqrt', 'log2']
max_depth = [int(x) for x in np.linspace(10, 100, num = 10)]
max_depth.append(None)
min_samples_split = [5, 10, 15, 20]
min_samples_leaf = [2, 4, 6, 8, 10]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt', 'log2'],
 'min_samples_leaf': [2, 4, 6, 8, 10],
 'min_samples_split': [5, 10, 15, 20],
 'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]}


In [0]:
model = RandomForestClassifier()
random_search = RandomizedSearchCV(estimator = model, param_distributions = random_grid, n_iter = 5, cv = 10, verbose = 2)
randomRorest_randomSearch = random_search.fit(X_train, y_train)
joblib.dump(randomRorest_randomSearch, 'randomForest_randomSearch.sav')

print("\nParameters:")
pprint(randomRorest_randomSearch.get_params())

predict(randomRorest_randomSearch, X_valid, y_valid)

In [0]:
params = {'n_estimators': [10, 15, 20, 25, 30, 40, 50],
          'max_features': ['auto'],
          'max_depth': [None],
          'min_samples_split': [2, 4, 6, 8, 10],
          'min_samples_leaf': [1, 2, 3, 4, 5, 6],
          'bootstrap': [True],
          'random_state': [None]}

model = RandomForestClassifier()
grid_search = GridSearchCV(estimator = model, param_grid = params, cv = 10, verbose = 2)
randomRorest_gridSearch = grid_search.fit(X_train, y_train)
joblib.dump(randomRorest_gridSearch, 'randomForest_gridSearch.sav')

print("\nParameters:")
pprint(randomRorest_gridSearch.get_params())

predict(randomRorest_gridSearch, X_valid, y_valid)

In [0]:
model = RandomForestClassifier()
randomForest_basic = randomForest.fit(X_train, y_train)
joblib.dump(randomForest_basic, 'randomForest_basic.sav')

print("\nParameters:")
pprint(randomForest_basic.get_params())

predict(randomForest_basic, X_valid, y_valid)

In [0]:
randomForest = RandomForestClassifier(class_weight=class_weights, n_estimators=len(Counter(Y)), criterion='entropy', random_state=0, n_jobs=len(Counter(Y)), max_depth=None, bootstrap=False)

selected_features = SelectFromModel(randomForest)
selected_features.fit(X_train, y_train)

features = X_train.columns[(selected_features.get_support())]
print("Selected features: {}".format(features))

X_train_important_features = selected_features.transform(X_train)
X_test_important_features = selected_features.transform(X_test)